# 07 Regression dengan KNN (K Nearest Neighbours)
* KNN adalah model machine learning yang dapat digunakan untuk melakukan prediksi berdasarkan kedekatan karakteristik dengan sejumlah tetangga terdekat.
* Prediksi yang dilakukan dapat diterapkan baik pada classification maupun regression tasks.

Referensi: https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm

## Sample Dataset

In [1]:
import pandas as pd

sensus = {'tinggi': [158, 170, 183, 191, 155, 163, 180, 158, 170], 
          'jk': ['pria', 'pria', 'pria', 'pria', 'wanita', 'wanita', 'wanita', 'wanita', 'wanita'],
          'berat': [64, 86, 84, 80, 49, 59, 67, 54, 67]}

sensus_df = pd.DataFrame(sensus)
sensus_df

,tinggi,jk,berat
0,158,pria,64
1,170,pria,86
2,183,pria,84
3,191,pria,80
4,155,wanita,49
5,163,wanita,59
6,180,wanita,67
7,158,wanita,54
8,170,wanita,67


Code diatas kita sudah menyiapkan suatu dictionary yang terdiri dari tiga buah keys yaitu tinggi, JK, dan berat. Key tinggi akan berasosiasi dengan sekumpulan nilai tinggi badan, JK akan berasosiasi dengan sekumpulan nilai jenis kelamin, dalam hal ini jenis kelaminnya terdiri dari dua nilai string yaitu pria dan wanita, lalu key berat akan berasosiasi dengan sekumpulan nilai berat badan. Dictionary ini akan ditampung ke dalam variable yang diberi nama sensus yang dimana selanjutnya akan digunakan sebagai dasar pembentukan pandas dataframe oleh karena itu code diatas melakakukan pemanggilan pd.DataFrame(sensus).

## Regression dengan KNN

### Features & Target

In [2]:
import numpy as np

X_train = np.array(sensus_df[['tinggi', 'jk']])
y_train = np.array(sensus_df['berat'])

print(f'X_train:\n{X_train}\n')
print(f'y_train: {y_train}')

X_train:
[[158 'pria']
 [170 'pria']
 [183 'pria']
 [191 'pria']
 [155 'wanita']
 [163 'wanita']
 [180 'wanita']
 [158 'wanita']
 [170 'wanita']]

y_train: [64 86 84 80 49 59 67 54 67]


Code diatas untuk data tinggi badan dan jenis kelamin akan dikonversikan ke dalam numpy array dan akan ditampung ke dalam variable X_train sebagai sekumpulan features untuk training set lalu untuk data berat badan juga akan dikonversikan menjadi numpy array untuk ditampung kedalam variable y_train sebagai sekumpulan nilai target untuk training set.

Code diatas memanfaatkan KNN untuk melakukan prediksi berat badan berdasarkan data tinggi badan dan jenis kelaminnya, karena disini yang diprediksi berupa nilai continous dan bukan kategori maka ini akan termasuk dalam regression task.

### Preprocess Dataset: Konversi Label menjadi Numerik Biner

In [3]:
X_train_transposed = np.transpose(X_train)

print(f'X_train:\n{X_train}\n')
print(f'X_train_transposed:\n{X_train_transposed}')

X_train:
[[158 'pria']
 [170 'pria']
 [183 'pria']
 [191 'pria']
 [155 'wanita']
 [163 'wanita']
 [180 'wanita']
 [158 'wanita']
 [170 'wanita']]

X_train_transposed:
[[158 170 183 191 155 163 180 158 170]
 ['pria' 'pria' 'pria' 'pria' 'wanita' 'wanita' 'wanita' 'wanita'
  'wanita']]


Proses transpose ini pada dasarnya akan mengubah posisi baris menjadi kolom dan posisi kolom menjadi baris. Posisi baris di atas menunjukkan instance sedangkan posisi kolomnya menunjukkan features. Maka kasus di atas mengubah posisi features menjadi baris dan bukan kolom.

Jika dilihat pada X_train_transposed terdiri dari 2 baris saja, nilai dari baris pertamanya akan berkorelasi dengan fitur tinggi badan, sedangkan untuk baris keduanya akan merepresentasikan sekumpulan nilai untuk jenis kelamin.

In [4]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
jk_binarised = lb.fit_transform(X_train_transposed[1])

print(f'jk: {X_train_transposed[1]}\n')
print(f'jk_binarised:\n{jk_binarised}')

jk: ['pria' 'pria' 'pria' 'pria' 'wanita' 'wanita' 'wanita' 'wanita' 'wanita']

jk_binarised:
[[0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]]


LabelBinarizer disini digunakan untuk mengkonversikan nilai pria dan wanita menjadi nilai biner 0 dan 1. Proses transformasi LabelBinarizer akan diterapkan pada jenis kelamin oleh karena itu dilakukan pemanggilan lb.fit_transform dan diterapkan pada X_train_transposed indeks ke 1, karena indeks ke 0 nya adalah tinggi badan sedangkan indeks ke 1 nya adalah jenis kelamin, lalu hasil transform nya akan ditampung kedalam variable jk_binarised.

In [5]:
jk_binarised = jk_binarised.flatten()
jk_binarised

array([0, 0, 0, 0, 1, 1, 1, 1, 1])

Proses flatten diatas digunakan untuk mengkonversikan multi dimensional array menjadi single dimensional array atau dengan kata lain method flatten ini digunakan untuk mengkonversikan array multi dimensi menjadi array dimensi tunggal.

In [6]:
X_train_transposed[1] = jk_binarised
X_train = X_train_transposed.transpose()

print(f'X_train_transposed:\n{X_train_transposed}\n')
print(f'X_train:\n{X_train}')

X_train_transposed:
[[158 170 183 191 155 163 180 158 170]
 [0 0 0 0 1 1 1 1 1]]

X_train:
[[158 0]
 [170 0]
 [183 0]
 [191 0]
 [155 1]
 [163 1]
 [180 1]
 [158 1]
 [170 1]]


X_train_transposed diatas akan di transpose balik agar yang tadinya baris kembali menjadi kolom yang tadinya kolom akan menjadi baris dan hasil transpose nya akan dikembalikan kedalam variable X_train

### Training KNN Regression Model

In [7]:
from sklearn.neighbors import KNeighborsRegressor

K = 3
model = KNeighborsRegressor(n_neighbors=K)
model.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=3)

Model machine learning yang akan digunakan adalah KNN dan yang akan di import adalah KNeighborsRegressor karena akan menggunakan KNN untuk regression tasks.

### Prediksi Berat Badan

In [8]:
X_new = np.array([[155, 1]])
X_new

array([[155,   1]])

Selanjutnya train model yang sebelumnya akan digunakan untuk melakukan prediksi berat badan berdasarkan data tinggi badan dan jenis kelamin

In [9]:
y_pred = model.predict(X_new)
y_pred

array([55.66666667])

Selanjutnya kita akan melakukan prediksi berat badan dengan memanfaatkan model KNeighborsRegressor yang sudah di training sebelumnya. Untuk hasilnya bisa dikatakan bahwa dengan tinggi badan 155cm dan jenis kelamin wanita diprediksi oleh model KNeighborsRegressor memiliki berat badan 55.66kg

### Evaluasi KNN Regression Model

In [10]:
X_test = np.array([[168, 0], [180, 0], [160, 1], [169, 1]])
y_test = np.array([65, 96, 52, 67])

print(f'X_test:\n{X_test}\n')
print(f'y_test: {y_test}')

X_test:
[[168   0]
 [180   0]
 [160   1]
 [169   1]]

y_test: [65 96 52 67]


Disini akan menggunakan beberapa matriks untuk mengukur performa dari model machine learning untuk kasus Regression task tapi sebelumnya kita harus menyiapkan testing set nya. Code diatas sudah disiapkan 4 buah data point, Pertama siapkan dulu sekumpulan nilai features untuk testing set nya kemudian dimasukkan kedalam variable X_test selain itu juga perlu disiapkan sekumpulan nilai target untuk kita tampung kedalam variable y_test.

In [11]:
y_pred = model.predict(X_test)
y_pred

array([70.66666667, 79.        , 59.        , 70.66666667])

Setelah testing set nya siap selanjutnya akan dilakukan prediksi terhadap testing set dengan memanfaatkan model KNN Regessor yang sudah di training sebelumnya

#### Coefficient of Determination atau $R^2$
Referensi: https://en.wikipedia.org/wiki/Coefficient_of_determination

In [12]:
from sklearn.metrics import r2_score

r_squared = r2_score(y_test, y_pred)

print(f'R-squared: {r_squared}')

R-squared: 0.6290565226735438


jika R-squared semakin mendekati 1 maka semakin baik dan jika R-squared semakin mendekati 0 atau bahkan nilainya negatif maka mengindikasikan bahwa modelnya kurang baik.

#### Mean Absolute Error (MAE) atau Mean Absolute Deviation (MAD)
$MAE$ is the average of the absolute values of the errors of the predictions.

$MAE = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|$

Referensi: https://en.wikipedia.org/wiki/Mean_absolute_error

In [13]:
from sklearn.metrics import mean_absolute_error

MAE = mean_absolute_error(y_test, y_pred)

print(f'MAE: {MAE}')

MAE: 8.333333333333336


Mean Absolute Error atau yang biasa disebut sebagai Mean Absolute Deviation adalah nilai rata-rata dari absolute error dari prediksi. MAE akan menghiting selisih atau error antara 𝑦𝑖 dengan 𝑦̂𝑖, 𝑦𝑖 ini merepresentasikan setiap nilai target pada testing set sedangkan 𝑦̂𝑖 ini merupakan nilai prediksi yang dihasilkan oleh model kita. Proses perhitungan selisih ini akan memungkinkan saja untuk menghasilkan nilai positif atau negatif, jika hasil prediksinya ternyata lebih kecil dari apa yang seharusnya maka nilainya positif, tetapi jika hasil prediksinya ternyata lebih besar dari yang seharusnya maka nilainya akan menjadi negatif, untuk menghindari nilai negatif maka harus menerapkan yang namanya absolute function yang berfungsi untuk menghilangkan nilai negatif.

#### Mean Squared Error (MSE) atau Mean Squared Deviation (MSD)
$MSE$ is the average of the squares of the errors of the predictions.

$MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$

Referensi: https://en.wikipedia.org/wiki/Mean_squared_error

In [14]:
from sklearn.metrics import mean_squared_error

MSE = mean_squared_error(y_test, y_pred)

print(f'MSE: {MSE}')

MSE: 95.8888888888889


Mean Squared Error atau yang biasa disebut dengan Mean Squared Deviation akan menghitung selisih atau error antara 𝑦𝑖 dengan 𝑦̂𝑖. 𝑦𝑖 merupakan nilai target dari testing set, sedangkan 𝑦̂𝑖 merupakan nilai estimasi yang dihasilkan oleh model kita. Untuk menghindari kemunculan nilai negatif pas MSE selisih nilainya akan dipangkatkan 2.

### Permasalahan Scaling pada Features

In [15]:
from scipy.spatial.distance import euclidean

# tinggi dalam milimeter
X_train = np.array([[1700, 0], [1600, 1]])
X_new = np.array([[1640, 0]])

[euclidean(X_new[0], d) for d in X_train]

[60.0, 40.01249804748511]

Untuk percobaan pertama ini tinggi badannya diukut dalam satuan mm, dataset yang ada memiliki 2 features yaitu tinggi badan dan jenis kelamin, nilai tinggi badan direpresentasikan dalam satuan cm, sedangkan jenis kelamin direpresentasikan dalam bilangan biner yaitu 0 dan 1.

Pada hasilnya bisa dilihat bahwa datapoint yang baru lebih dekat dengan datapoint kedua bila dibandingkan dengan datapoint pertama.

In [16]:
# tinggi dalam meter
X_train = np.array([[1.7, 0], [1.6, 1]])
X_new = np.array([[1.64, 0]])

[euclidean(X_new[0], d) for d in X_train]

[0.06000000000000005, 1.0007996802557442]

Untuk percobaan kedua ini tinggin badannya diukur dalam satuan m

Pada hasilnya bisa dilihat bahwa datapoint yang baru lebih dekat dengan datapoint pertama bila dibandingkan dengan datapoint kedua

Kesimpulannya terdapat ketidakkonsistenan dalam pengukuran jarak padahal telah menggunakan data set yang sama persis yang membedakan hanya pada satuan pengukuran saja

### Menerapkan Standard Scaler (Standard Score atau Z-Score)
Standardize features by removing the mean and scaling to unit variance.

$z = \frac{x - \bar{x}}{S}$

Referensi: https://en.wikipedia.org/wiki/Standard_score

In [17]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

* ${x}$ = mempresentasikan nilai features
* $\bar{x}$ = mempresentasikan rata-rata nilai features
* ${S}$ = standard deviation dari sekumpulan nilai features

In [18]:
# tinggi dalam milimeter
X_train = np.array([[1700, 0], [1600, 1]])
X_train_scaled = ss.fit_transform(X_train)
print(f'X_train_scaled:\n{X_train_scaled}\n')

X_new = np.array([[1640, 0]])
X_new_scaled = ss.transform(X_new)
print(f'X_new_scaled: {X_new_scaled}\n')

jarak = [euclidean(X_new_scaled[0], d) for d in X_train_scaled]
print(f'jarak: {jarak}')

X_train_scaled:
[[ 1. -1.]
 [-1.  1.]]

X_new_scaled: [[-0.2 -1. ]]

jarak: [1.2, 2.1540659228538015]


Hasil dari code diatas bisa dilihat bahwa nilai-nilai nya sudah tidak lagi menggunakan satuan cm, mm ataupun m, satuannya sudah menggunakan satuan standar skor atau z-score.

In [19]:
# tinggi dalam meter
X_train = np.array([[1.7, 0], [1.6, 1]])
X_train_scaled = ss.fit_transform(X_train)
print(f'X_train_scaled:\n{X_train_scaled}\n')

X_new = np.array([[1.64, 0]])
X_new_scaled = ss.transform(X_new)
print(f'X_new_scaled: {X_new_scaled}\n')

jarak = [euclidean(X_new_scaled[0], d) for d in X_train_scaled]
print(f'jarak: {jarak}')

X_train_scaled:
[[ 1. -1.]
 [-1.  1.]]

X_new_scaled: [[-0.2 -1. ]]

jarak: [1.2000000000000026, 2.1540659228538006]


Dari hasil eksperimen kedua ini menghasilkan jarak yang sama atau setidaknya sangat mendekati dengan nilai jarak pada eksperimen pertama, disini juga bisa dilihat bahwa datapoint baru juga lebih dekat dengan datapoint pertama bila dibandingkan dengan datapoint kedua.

### Menerapkan Features Scaling pada KNN

#### Dataset

In [20]:
# Training Set
X_train = np.array([[158, 0], [170, 0], [183, 0], [191, 0], [155, 1], [163, 1],
                    [180, 1], [158, 1], [170, 1]])

y_train = np.array([64, 86, 84, 80, 49, 59, 67, 54, 67])

# Test Set
X_test = np.array([[168, 0], [180, 0], [160, 1], [169, 1]])
y_test = np.array([65, 96, 52, 67])

Kita akan lihat bagaimana features scaling akan berpotensi dalam meningkatkan performa dari model KNN. Features Scaling yang akan digunakan adalah Standard Scaler.

#### Features Scaling (Standard Scaler)

In [21]:
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

print(f'X_train_scaled:\n{X_train_scaled}\n')
print(f'X_test_scaled:\n{X_test_scaled}\n')

X_train_scaled:
[[-0.9908706  -1.11803399]
 [ 0.01869567 -1.11803399]
 [ 1.11239246 -1.11803399]
 [ 1.78543664 -1.11803399]
 [-1.24326216  0.89442719]
 [-0.57021798  0.89442719]
 [ 0.86000089  0.89442719]
 [-0.9908706   0.89442719]
 [ 0.01869567  0.89442719]]

X_test_scaled:
[[-0.14956537 -1.11803399]
 [ 0.86000089 -1.11803399]
 [-0.82260955  0.89442719]
 [-0.06543485  0.89442719]]



Hasil code diatas merupakan X_train yang sudah di scaling dan X_test yang sudah di scaling 

#### Training & Evaluasi Model

In [22]:
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

MAE = mean_absolute_error(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)

print(f'MAE: {MAE}')
print(f'MSE: {MSE}')

MAE: 7.583333333333336
MSE: 85.13888888888893


Jika dibandingkan nilainya dengan hasil evaluasi model sebelumnya, hasil nya lebih baik dari sebelumnya, Nilai MAE dan MSE setelah menerapkan fitur scaling bisa dilihat bahwa hasilnya lebih kecil atau dengan kata lain kita bisa menghasilkan model dengan kualitas atau performa yang lebih baik setelah menerapkan fitur scaling.